In [1]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = 'drive/My Drive/Bangkit Capstone PT CJNN/ML/'
SAMSUM_ID_PATH = PROJECT_ROOT + 'dataset/emotion_detection/'

Mounted at /content/drive


In [2]:
!pip install --upgrade transformers
!pip install simpletransformers
!pip install ipython-autotime
%load_ext autotime

import os, sys
import pandas as pd
import tensorflow as tf

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, TFTrainer, TFTrainingArguments
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 41.8MB/s 
     |████████████████████████████████| 901kB 22.5MB/s 
     |████████████████████████████████| 225kB 6.8MB/s 
     |████████████████████████████████| 122kB 10.1MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 8.2MB 11.9MB/s 
     |████████████████████████████████| 1.8MB 47.9MB/s 
     |████████████████████████████████| 235kB 52.5MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 1.2MB 40.8MB/s 
     |████████████████████████████████| 174kB 57.6MB/s 
     |████████████████████████████████| 4.2MB 43.0MB/s 
     |████████████████████████████████| 112kB 49.8MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 133kB 41.3MB/s 
     |████████████████████████████████| 112kB 45.6MB/s 


time: 5.33 s (started: 2021-05-29 10:20:32 +00:00)


In [3]:
cd drive/My Drive/Bangkit Capstone PT CJNN/ML/dataset/emotion_detection/

/content/drive/.shortcut-targets-by-id/1wTsiCh91wXfDb-VjsqGVzrBbIemDL0jD/Bangkit Capstone PT CJNN/ML/dataset/emotion_detection
time: 956 ms (started: 2021-05-29 10:20:37 +00:00)


In [4]:
!ls

development	     test_emotion.csv	   train_emotion_final.csv
dev_emotion.csv      test_emotion_fix.csv  train_emotion_fix.csv
dev_emotion_fix.csv  train		   Twitter_Emotion_Dataset.csv
test		     train_emotion.csv
time: 319 ms (started: 2021-05-29 10:20:38 +00:00)


## Load Data

In [ ]:
dataset = pd.read_csv('Twitter_Emotion_Dataset.csv')

time: 33.4 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
dataset

,label,tweet
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata..."
...,...,...
4396,love,"Tahukah kamu, bahwa saat itu papa memejamkan m..."
4397,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...
4398,anger,"5. masa depannya nggak jelas. lha iya, gimana ..."
4399,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...


time: 23.4 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
dataset['label'].value_counts()

anger      1101
happy      1017
sadness     997
fear        649
love        637
Name: label, dtype: int64

time: 11.7 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
dataset.query('label == "love"')

,label,tweet
16,love,Setiap kesempatan yg pernah hadir tuk dapat me...
24,love,kan kupeluk engkau erat2 hingga tak ada seoran...
35,love,Kenapa pasangan seringkali diibaratkan sebagai...
49,love,Ku rindu bisik mu di telingaku Seraya kau berk...
93,love,halohaa selamat siang Denise Udah pulang sekol...
...,...,...
4385,love,[USERNAME] Saya ngeliat rp cewe bukan dari cha...
4386,love,"[USERNAME] Apaa eh sengajanya tu , sayang dah ..."
4388,love,Membuat kita dapat secara tidak langsung menya...
4390,love,aku sayang skz karena mereka super talented da...


time: 34 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
tdf = dataset[dataset.label != "love"]

time: 5.21 ms (started: 2021-05-17 12:16:23 +00:00)


In [ ]:
df

,label,tweet
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata..."
...,...,...
4395,sadness,nah diam lebih bagus yaudah saya diam saja deh...
4397,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...
4398,anger,"5. masa depannya nggak jelas. lha iya, gimana ..."
4399,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...


time: 22.1 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
ddf = pd.read_csv('train/anger-train-translated-0.csv', header=None)

time: 10.1 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
for j in ['anger', 'joy']:
  for i in range(1, 9):
    file = 'train/' + j + '-train-translated-' + str(i) + '.csv'
    df = pd.read_csv(file, header=None)
    ddf = ddf.append(df, ignore_index = True)

time: 107 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
ddf

,0,1,2,3,4
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938,Bagaimana fu * k! Siapa sih! memindahkan lemar...
1,10001,So my Indian Uber driver just called someone t...,anger,0.896,Jadi pengemudi Uber India saya baru saja menye...
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896,@DPD_UK Saya minta parsel saya dikirim ke toko...
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896,jadi ef mana pun yang menyeka pantat yang mena...
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896,Jangan bergabung dengan @BTCare mereka meletak...
...,...,...,...,...,...
1575,30818,It's just the lack of company and liveliness o...,joy,0.058,Hanya kurangnya teman dan keaktifan di sini ya...
1576,30819,Quinn's short hair makes me sad. #glee,joy,0.040,Rambut pendek Quinn membuatku sedih. #lagu
1577,30820,hate overthinking e v e r y t h i n g like i j...,joy,0.040,benci terlalu banyak berpikir seperti aku hany...
1578,30821,People who cheer for sports teams completely o...,joy,0.020,Orang-orang yang mendukung tim olahraga di lua...


time: 30.3 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
for i in range(12):
    file = 'train/fear-train-translated-' + str(i) + '.csv'
    df = pd.read_csv(file, header=None)
    ddf = ddf.append(df, ignore_index = True)

time: 102 ms (started: 2021-05-17 12:10:44 +00:00)


In [ ]:
for i in range(8):
    file = 'train/sadness-train-translated-' + str(i) + '.csv'
    df = pd.read_csv(file, header=None)
    ddf = ddf.append(df, ignore_index = True)

time: 68.9 ms (started: 2021-05-17 12:10:45 +00:00)


In [ ]:
ddf

,0,1,2,3,4
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938,Bagaimana fu * k! Siapa sih! memindahkan lemar...
1,10001,So my Indian Uber driver just called someone t...,anger,0.896,Jadi pengemudi Uber India saya baru saja menye...
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896,@DPD_UK Saya minta parsel saya dikirim ke toko...
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896,jadi ef mana pun yang menyeka pantat yang mena...
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896,Jangan bergabung dengan @BTCare mereka meletak...
...,...,...,...,...,...
3508,40781,@VivienLloyd Thank you so much! Just home - st...,sadness,0.104,@VivienLloyd Terima kasih banyak! Baru pulang ...
3509,40782,Just put the winter duvet on ☃️❄️🌬☔️,sadness,0.104,Cukup letakkan selimut musim dingin di ☃️❄️🌬☔️
3510,40783,@SilkInSide @TommyJoeRatliff that's so pretty!...,sadness,0.088,@SilkInSide @TommyJoeRatliff itu sangat cantik...
3511,40784,@BluesfestByron second artist announcement loo...,sadness,0.083,Pengumuman artis kedua @BluesfestByron terliha...


time: 27.5 ms (started: 2021-05-17 12:10:45 +00:00)


In [ ]:
ddf_final = ddf[[2,4]]

time: 3.91 ms (started: 2021-05-17 12:13:12 +00:00)


In [ ]:
ddf_final

,2,4
0,anger,Bagaimana fu * k! Siapa sih! memindahkan lemar...
1,anger,Jadi pengemudi Uber India saya baru saja menye...
2,anger,@DPD_UK Saya minta parsel saya dikirim ke toko...
3,anger,jadi ef mana pun yang menyeka pantat yang mena...
4,anger,Jangan bergabung dengan @BTCare mereka meletak...
...,...,...
3508,sadness,@VivienLloyd Terima kasih banyak! Baru pulang ...
3509,sadness,Cukup letakkan selimut musim dingin di ☃️❄️🌬☔️
3510,sadness,@SilkInSide @TommyJoeRatliff itu sangat cantik...
3511,sadness,Pengumuman artis kedua @BluesfestByron terliha...


time: 22.1 ms (started: 2021-05-17 12:13:17 +00:00)


In [ ]:
ddf_final.columns = ['label', 'sentence']

time: 1.81 ms (started: 2021-05-17 12:16:49 +00:00)


In [ ]:
tdf.columns = ['label', 'sentence']

time: 1.93 ms (started: 2021-05-17 12:17:03 +00:00)


In [ ]:
final = ddf_final.append(tdf, ignore_index = True)

time: 6.59 ms (started: 2021-05-17 12:17:24 +00:00)


In [ ]:
final

,label,sentence
0,anger,Bagaimana fu * k! Siapa sih! memindahkan lemar...
1,anger,Jadi pengemudi Uber India saya baru saja menye...
2,anger,@DPD_UK Saya minta parsel saya dikirim ke toko...
3,anger,jadi ef mana pun yang menyeka pantat yang mena...
4,anger,Jangan bergabung dengan @BTCare mereka meletak...
...,...,...
7272,sadness,nah diam lebih bagus yaudah saya diam saja deh...
7273,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...
7274,anger,"5. masa depannya nggak jelas. lha iya, gimana ..."
7275,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...


time: 23.8 ms (started: 2021-05-17 12:17:29 +00:00)


In [ ]:
final.to_csv('train_emotion.csv',  index=False, header=True)

time: 80.2 ms (started: 2021-05-17 12:28:07 +00:00)


In [ ]:
!ls

development  test  train  train_emotion.csv  Twitter_Emotion_Dataset.csv
time: 149 ms (started: 2021-05-17 12:28:13 +00:00)


In [ ]:
/content/drive/MyDrive/Bangkit Capstone PT CJNN/ML/dataset/emotion_detection/train_emotion.csv